In [2]:
import pandas as pd

In [ ]:
def process_and_concat_sheets(file_path):
    # Load the Excel file
    excel_file = pd.ExcelFile(file_path)
    
    # List to store processed DataFrames
    all_data = []
    print(' excel_file.sheet_names',excel_file.sheet_names)
    # Loop through all sheet names
    for sheet_name in excel_file.sheet_names:
        print('Processing sheet:', sheet_name)
        
        # Read the sheet and specify column types
        file = pd.read_excel(file_path, sheet_name=sheet_name, dtype={'KB J CAP': str})
        if "Mpay" in sheet_name: 
            # Extract and create the necessary columns
            file['Loan No.'] = file.iloc[:, 10]  # Assuming 'Loan No.' is in the 8th column (index 7)
            file['Loan No.'] = file['Loan No.'].astype(str)
            file['Payment'] = file.iloc[:, 5]  # Assuming 'Payment' is in the 16th column (index 15)
            file['Type'] = sheet_name  # Set the Type column to the sheet name
            file['Date'] = file.iloc[:, 3]  # Assuming 'Date' is in the 5th column (index 4)
            file['Date'] = file['Date'].astype(str)
        else :
            file['Loan No.'] = file.iloc[:, 7]  # Assuming 'Loan No.' is in the 8th column (index 7)
            file['Loan No.'] = file['Loan No.'].astype(str)
            file['Payment'] = file.iloc[:, 15]  # Assuming 'Payment' is in the 16th column (index 15)
            file['Type'] = sheet_name  # Set the Type column to the sheet name
            file['Date'] = file.iloc[:, 4]  # Assuming 'Date' is in the 5th column (index 4)
            file['Date'] = file['Date'].astype(str)

        
        # Optionally print the processed file (for debugging)
        print('Processed data from', sheet_name, file.head())
        file = file[['Loan No.',
       'Payment', 'Type', 'Date']]
        # Append the processed file to the list
        all_data.append(file)
    
    # Concatenate all the DataFrames in the list into a single DataFrame
    concatenated_data = pd.concat(all_data, ignore_index=True)
    
    return concatenated_data

In [19]:
import glob
file_list = glob.glob("./file/*")

In [20]:

file_list = glob.glob(f"./file/*")
# file_list = glob.glob(f"./file/*")


# List to store concatenated data from each file
all_files_data = []

# Loop through each file and process it
for file_path in file_list:
    print(f"Processing file: {file_path}")
    
    # Process the sheets in each file and concatenate the results
    file_data = process_and_concat_sheets(file_path)
    
    # Append the data to the list
    all_files_data.append(file_data)

# Concatenate all file data into a single DataFrame
final_data = pd.concat(all_files_data, ignore_index=True)

print(final_data)

Processing file: ./file\5DA4F010
 excel_file.sheet_names ['BBL', 'CS', 'M pay']
Processing sheet: BBL
Processed data from BBL    H  000001  002    1353097056  KB J CAP    ITAL C  \
0  D     2.0  2.0  1.353097e+09  17102024  230332.0   
1  D     3.0  2.0  1.353097e+09  17102024  230407.0   
2  D     4.0  2.0  1.353097e+09  17102024  230412.0   
3  D     5.0  2.0  1.353097e+09  17102024  230445.0   
4  D     6.0  2.0  1.353097e+09  17102024  230518.0   

  OMPANY LIMITED            18102024KBJCAP        Unnamed: 7     Unnamed: 8  \
0                    นาย ธนวัฒน์ ศรีโพนทอง  7240770103299756    14242596563   
1                        ORNJIRA WANNAKOOL  7231270101833308    12170043692   
2                            สุภาภรณ์ สาโท  7240170102026268    14241697171   
3                       นาย วรัชญ์ ยั่งยืน  7240770103413207    14242675803   
4                          รัชนี ริยาพันธ์  7230230100198458  1829900160631   

   Unnamed: 9  ... Unnamed: 11  Unnamed: 12 Unnamed: 13 Unnamed: 14 

IndexError: single positional indexer is out-of-bounds

In [17]:
final_data

NameError: name 'final_data' is not defined

In [21]:
def check_sheets_with_mpay(file_path):
    # Load the Excel file
    excel_file = pd.ExcelFile(file_path)

    # List all sheet names
    sheet_names = excel_file.sheet_names
    print("All sheet names:", sheet_names)

    # Filter sheets that contain 'Mpay' (case-insensitive)
    mpay_sheets = [sheet for sheet in sheet_names if "mpay" in sheet.lower()]

    if mpay_sheets:
        print("Sheets containing 'Mpay':", mpay_sheets)
    else:
        print("No sheets contain 'Mpay'.")

    return mpay_sheets  # Return list of matching sheet names

# Example usage
file_path = "your_excel_file.xlsx"
mpay_sheets = check_sheets_with_mpay(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'your_excel_file.xlsx'

In [51]:
final_data.to_excel('./output/0524.xlsx')


In [ ]:
def split_and_save_data(data, output_file):
    # Split data into chunks of 500,000 rows if needed
    chunk_size = 500000
    num_chunks = (len(data) // chunk_size) + (1 if len(data) % chunk_size > 0 else 0)
    
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        # Write each chunk to a separate sheet
        for chunk_num in range(num_chunks):
            start_row = chunk_num * chunk_size
            end_row = min((chunk_num + 1) * chunk_size, len(data))
            chunk = data.iloc[start_row:end_row]
            
            # Save the chunk to a new sheet
            chunk.to_excel(writer, sheet_name=f'Sheet_{chunk_num + 1}', index=False)
            print(f"Saved Sheet_{chunk_num + 1} with rows {start_row} to {end_row}")

In [ ]:
output_file = './output/0524.xlxs'
if len(final_data) > 500000:
        print(f"Total rows {len(final_data)} exceed 500,000. Splitting into multiple sheets.")
        split_and_save_data(final_data, output_file)
else:
    # If it's less than 500,000 rows, save it as one sheet
    final_data.to_excel(output_file, index=False, sheet_name='Sheet_1')
    print(f"Saved all data to {output_file}.")